# Document Question-Answer [h2oGPT](https://github.com/h2oai/h2ogpt)

In this notebook, we demonstrate how one can use h2oGPT with a large language model.

To begin, please get free ngrok account to get auth token (e.g.) using your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup .  You will be asked for this token below in an input box.

In [1]:
!git clone https://github.com/h2oai/h2ogpt.git
!cd h2ogpt && git checkout 2668694581347b0d1afe76760213db46f7214126 -q
!cp -ar h2ogpt/. ./
!rm -r h2ogpt

Cloning into 'h2ogpt'...
remote: Enumerating objects: 19844, done.
remote: Counting objects: 100% (3773/3773), done.
remote: Compressing objects: 100% (385/385), done.
remote: Total 19844 (delta 3503), reused 3540 (delta 3382), pack-reused 16071
Receiving objects: 100% (19844/19844), 34.14 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (14189/14189), done.


In [2]:
# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

In [ ]:
# Install dependencies
!for fil in requirements.txt reqs_optional/requirements_optional_langchain.txt reqs_optional/requirements_optional_gpt4all.txt reqs_optional/requirements_optional_langchain.gpllike.txt reqs_optional/requirements_optional_langchain.urls.txt ; do pip install -r $fil ; done


  Cloning https://github.com/huggingface/peft.git (to revision 0b62b4378b4ce9367932c73540349da9a41bdea8) to /tmp/pip-req-build-te4fz4o7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-te4fz4o7
  Running command git rev-parse -q --verify 'sha^0b62b4378b4ce9367932c73540349da9a41bdea8'
  Running command git fetch -q https://github.com/huggingface/peft.git 0b62b4378b4ce9367932c73540349da9a41bdea8
  Running command git checkout -q 0b62b4378b4ce9367932c73540349da9a41bdea8
  Resolved https://github.com/huggingface/peft.git to commit 0b62b4378b4ce9367932c73540349da9a41bdea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Ignoring pypandoc: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Sign-up for free ngrok account using (e.g.) your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup

!pip install pyngrok
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open an http ngrok tunnel
connection_string = ngrok.connect(7860, "http").public_url
print("Once server is up and says Running on local URL:  http://0.0.0.0:7860, click on this link, then click on Visit Site: %s" % connection_string)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=23dcf1ef7e83a97022a720714e5a1037b13abedd47aa8cef7bf580fc9f1e9f8f
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


Once server is up and says Running on local URL:  http://0.0.0.0:7860, click on this link, then click on Visit Site: https://55cd-34-124-161-248.ngrok-free.app


In [ ]:
!GRADIO_SERVER_PORT=7860 python generate.py --base_model=togethercomputer/RedPajama-INCITE-Chat-3B-v1 --prompt_type=human_bot --score_model=None --langchain_mode=ChatLLM --visible_langchain_modes="['ChatLLM', 'UserData', 'MyData']" --user_path=user_path --share=False --hf_embedding_model=sentence-transformers/all-MiniLM-L6-v2

Using Model togethercomputer/redpajama-incite-chat-3b-v1
Prep: persist_directory=db_dir_ChatLLM does not exist, regenerating
Prep: persist_directory=db_dir_UserData does not exist, regenerating
0it [00:00, ?it/s]
0it [00:00, ?it/s]
Loaded 0 sources for potentially adding to UserData
2023-07-14 19:55:57.673554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
device_map: {'': 0}
Model {'base_model': 'togethercomputer/RedPajama-INCITE-Chat-3B-v1', 'tokenizer_base_model': '', 'lora_weights': '', 'inference_server': '', 'prompt_type': 'human_bot', 'prompt_dict': {'promptA': '', 'promptB': '', 'PreInstruct': '<human>: ', 'PreInput': None, 'PreResponse': '<bot>:', 'terminate_response': ['\n<human>:', '\n<bot>:', '<human>:', '<bot>:', '<bot>:'], 'chat_sep': '\n', 'chat_turn_sep': '\n', 'humanstr': '<human>:', 'botstr': '<bot>:', 'gen

In [ ]:
# can kill old ngrok + generate and try again
do_kill = False
if do_kill:
  !pkill -f generate --signal 9
  !pkill -f frpc_linux_amd --signal 9
  !pkill -f ngrok --signal 9